<a href="https://colab.research.google.com/github/Han-lim/Real-Fake-image-classification/blob/main/0601_EffNet04%2BXcpNet_soft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 현재까지의 best score 87.64 : 
EfficientNet7(Ensemble(Soft voting) + Xception Net + ... 에 대한 Hard voting)

------
##1. Optimizer, LR Scheduler 시행착오:  
 - Optimizer: adam, radam
 - stepLR, cosineAnnealingLR, custom cosine annealing warmup restarts(SGDR)

<strong> * 사용한 조합 : radam + cosineAnnealingLR </strong>
<br></br>

----
##2. Augmentation 시행착오:
- HorizontalFlip, Resize(128, 128)  
- cutout, GridDistortion 

<strong> * 다양한 augmentation을 시도해본 후, resize(128, 128)만 사용하는 것으로 결정함. </strong>
<br></br>

---
## 3. Model
- Efficient Net
- Xception net 

<strong>* 위의 두 모델을 중점으로 앙상블을 적용함 </strong>
<br></br>

-----
##4. dropout :
- efficient net(dropout:0.2), xception net(dropout: 0.5)
<strong> *  drop out은 모델별로 효과가 상이함 </strong>
<br></br>

-----

##5. 앙상블 방법 :
- soft voting


# 1. library 설치 및 import

In [1]:
from typing import Tuple, List, Sequence, Callable

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch

from torch import nn, Tensor
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset

!pip install -U git+https://github.com/albu/albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn.functional as F
from sklearn.model_selection import train_test_split

import time

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-1of1kren
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-1of1kren
  Created wheel for albumentations: filename=albumentations-1.0.0-cp37-none-any.whl size=98140 sha256=258bee4dc861682bf2db3880d9070b9e9b499dec456b7ae80d08fb1529d77eab
  Stored in directory: /tmp/pip-ephem-wheel-cache-pyjcc1p8/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Efficient Net 다운로드
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=45398777baa77049b1e931cf59ff8d7f554f1d5da23aafde8170b214494d0465
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [4]:
# Xception Net 다운로드
!pip install timm
import timm

     |████████████████████████████████| 348kB 8.1MB/s 


In [5]:
# optimizer 다운로드
!pip install torch_optimizer
import torch_optimizer as optim

     |████████████████████████████████| 81kB 7.1MB/s 


In [6]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

print('using device:', device)

using device: cuda:0


In [7]:
device = "cuda:0"
dtype = torch.float
ltype = torch.long # entropy

# 2. 데이터 불러오기

In [8]:
# fake 1, real 0 로 사용
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/통기계플젝/face_image/face_images.csv')
train_df.head()

,path,real,fake
0,./face_image/fake/JFH50GFJUL.jpg,0,1
1,./face_image/fake/0VPS5TI60G.jpg,0,1
2,./face_image/real/61911.jpg,1,0
3,./face_image/fake/APADHGXN31.jpg,0,1
4,./face_image/fake/SJO2UL69C2.jpg,0,1


In [9]:
print(train_df.shape)

(20000, 3)


In [10]:
# 경로 수정
tmp = train_df['path'].str[2:]
train_df['path'] = '/content/drive/MyDrive/Colab Notebooks/통기계플젝/'+tmp

In [11]:
train_df.head()

,path,real,fake
0,/content/drive/MyDrive/Colab Notebooks/통기계...,0,1
1,/content/drive/MyDrive/Colab Notebooks/통기계...,0,1
2,/content/drive/MyDrive/Colab Notebooks/통기계...,1,0
3,/content/drive/MyDrive/Colab Notebooks/통기계...,0,1
4,/content/drive/MyDrive/Colab Notebooks/통기계...,0,1


In [12]:
# 재현을 위한 seed 고정
import random
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# 3. 데이터 셋 구성

In [13]:
class FaceDataset(Dataset):
  def __init__(self, image_label, transforms) :
    self.df = image_label
    self.transforms = transforms 
        
  def __len__(self) -> int:
    return self.df.shape[0]

  def __getitem__(self, index: int) -> Tuple[Tensor]:
    assert index <= len(self), 'index range error' 
      
    image_dir = self.df.iloc[index, ]['path']  
    image_id = self.df.iloc[index, ]['fake'].astype(np.int64)
    
    image =  cv2.imread(image_dir, cv2.COLOR_BGR2RGB)
    target = torch.as_tensor(image_id, dtype=torch.long)

    if self.transforms is not None :
      image = self.transforms(image=image)['image']

    image = image/255.0  # numpy array를 0~1사이의 실수를 갖도록 변환
    
    return image, target

In [14]:
class TestDataset(Dataset):
  def __init__(self, image, transforms) :
    self.image = image
    self.transforms = transforms
        
  def __len__(self) -> int:
    return len(self.image)

  def __getitem__(self, index: int) -> Tuple[Tensor]:
    assert index <= len(self), 'index range error' 
    
    image_name = self.image[index]
    image_dir = '/content/drive/MyDrive/Colab Notebooks/통기계플젝/face_image/test/' + image_name

    image =  cv2.imread(image_dir, cv2.COLOR_BGR2RGB)
    
    if self.transforms is not None :
      image = self.transforms(image=image)['image']
    image = image/255.0

    return image_name, image

In [15]:
# Image Augmentation
transforms_tr = A.Compose([
    A.Resize(128, 128),
    ToTensorV2(), ])

transforms_val = A.Compose([
   A.Resize(128, 128),
    ToTensorV2(), ])

# 4. 모델 정의

In [ ]:
# efficient net 정의
class efficientnet(torch.nn.Module):
    def __init__(self):
        super(efficientnet, self).__init__()
        self.conv2d = nn.Conv2d(3, 3, 3, stride=1)
        self.effnet = EfficientNet.from_name('efficientnet-b4', dropout_rate=0.2) # 구조만 사용
        self.FC = nn.Linear(1000, 2)

        nn.init.xavier_normal_(self.FC.weight)

    def forward(self, x):
        x = F.silu(self.conv2d(x))

        # effnet을 추가
        x = F.silu(self.effnet(x))

        # 마지막 출력에 nn.Linear를 추가
        x = self.FC(x)
        return x

model = efficientnet()
model

efficientnet(
  (conv2d): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (effnet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 

In [16]:
# xception net 정의
class xceptionnet(torch.nn.Module):
    def __init__(self):
        super(xceptionnet, self).__init__()
        self.xcpnet = timm.create_model('xception', pretrained=False) # 구조만 사용
        self.FC = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.xcpnet(x)
        x = self.FC(x)
        return x

In [33]:
xceptionnet()

xceptionnet(
  (xcpnet): Xception(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplac

# Efficient Net train

In [ ]:
###########################################################################
# - n_splits = 10: train/valid set split을 9대 1로 하려고 설정            #
# - CUDA Memory 문제로 3fold 까지만 진행                                  #
# - train 이후에 런타임 초기화해서 저장한 best model들 다시 호출 후 test  #
#                                                                         #
###########################################################################
num_epochs = 30
EARLY_STOPPING_EPOCH = 6
SEED = 42

# cross validation을 적용하기 위해 KFold 생성
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)

# train_df에서 train_idx와 val_idx를 생성
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(train_df),1): # enumerate( , 1) 1부터 index 시작
  print('[fold: %d]' % fold_index)

  # cuda cache 초기화
  torch.cuda.empty_cache()

  #train fold, validation fold 분할
  train = train_df.iloc[trn_idx]
  valid  = train_df.iloc[val_idx]

  tr_dataset = FaceDataset(image_label=train, transforms=transforms_tr)
  val_dataset = FaceDataset(image_label=valid, transforms=transforms_val)

  train_loader = DataLoader(tr_dataset, batch_size=64, shuffle=True, num_workers=8)
  valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)

  # 모델 선언
  model = efficientnet()
  model.to(device)

  # optimizer : RAdam / lr_scheduler : CosineAnnealing
  optimizer = optim.RAdam(model.parameters(), lr=0.0015, betas=(0.9, 0.999), weight_decay=1e-4)
  lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


  # train 시작
  valid_early_stop = 0

  valid_best_loss = float('inf')
  since = time.time()

  final_train_loss = []
  final_train_acc = []
  final_valid_loss = []
  final_valid_acc = []

  for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    for i, (images, targets) in enumerate(train_loader) : 
      optimizer.zero_grad()
      
      images = images.to(device, dtype)
      targets = targets.to(device, ltype)
      model.train()
    
      scores = model(images)
      _, preds = scores.max(dim=1)

      loss = F.cross_entropy(scores, targets)
      loss.backward()
      optimizer.step()
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      train_loss_list.append(loss)
      train_acc_list.append(acc)

      if i % 20 == 0 :
        print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)
    
    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    valid_loss_list = []
    valid_acc_list = []
    for i, (images, targets) in enumerate(valid_loader) : 
      optimizer.zero_grad()
      images = images.to(device=device, dtype=dtype)
      targets = targets.to(device=device, dtype=ltype)
      model.eval()

      with torch.no_grad():
        scores = model(images)
        loss = F.cross_entropy(scores, targets)
        _, preds = scores.max(dim=1)
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      valid_loss_list.append(loss)
      valid_acc_list.append(acc)
  
    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
      valid_best_loss = val_mean_loss
      valid_early_stop = 0
      # new best model save (valid 기준)
      best_model = model
      path = '/content/drive/MyDrive/Colab Notebooks/통기계플젝/model/efficient/'
      torch.save({'model_state_dict' : best_model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict()}, f'{path}{fold_index}-fold_best-model.pth')

    else:
      # early stopping    
      valid_early_stop += 1
      if valid_early_stop >= EARLY_STOPPING_EPOCH:
        print("EARLY STOPPING!!")
        break

    lr_sched.step()
  best_models.append(best_model)
  if fold_index == 3:
    break

[fold: 1]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


 ====================== epoch 1 ======================
Iteration   0 | Train Loss  0.7075 | Classifier Accuracy 50.00
Iteration  20 | Train Loss  0.7088 | Classifier Accuracy 57.81
Iteration  40 | Train Loss  0.7213 | Classifier Accuracy 51.56
Iteration  60 | Train Loss  0.6994 | Classifier Accuracy 56.25
Iteration  80 | Train Loss  0.6788 | Classifier Accuracy 59.38
Iteration 100 | Train Loss  0.7087 | Classifier Accuracy 51.56
Iteration 120 | Train Loss  0.7461 | Classifier Accuracy 50.00
Iteration 140 | Train Loss  0.7721 | Classifier Accuracy 46.88
Iteration 160 | Train Loss  0.6641 | Classifier Accuracy 64.06
Iteration 180 | Train Loss  0.7192 | Classifier Accuracy 59.38
Iteration 200 | Train Loss  0.6302 | Classifier Accuracy 62.50
Iteration 220 | Train Loss  0.6038 | Classifier Accuracy 70.31
Iteration 240 | Train Loss  0.6099 | Classifier Accuracy 75.00
Iteration 260 | Train Loss  0.5982 | Classifier Accuracy 70.31
Iteration 280 | Train Loss  0.6298 | Classifier Accuracy 64.06


In [16]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/통기계플젝/face_image/submission.csv")
submission.head()

,image,label
0,test14200.jpg,NaN
1,test12178.jpg,NaN
2,test12713.jpg,NaN
3,test13712.jpg,NaN
4,test11739.jpg,NaN


In [15]:
import numpy as np
# softmax_0: 0으로 분류할 확률
def softmax_0(score0, score1) :
  exp_a0 = np.exp(score0)
  exp_a1 = np.exp(score1)
  sum_exp = exp_a0+exp_a1
  y = exp_a0 / sum_exp
  return y

# ensemble 시 threshold 설정을 위해 모든 score를 probability로 변환

# Efficient Net inference

In [ ]:
# inference에 사용할 model과 optimizer
model = efficientnet()
optimizer = optim.RAdam(model.parameters(), lr=0.0015, betas=(0.9, 0.999), weight_decay=1e-4)

prob_zero = [0 for _ in range(5000)]

for i in range(1, 4):
  checkpoint = torch.load(f'/content/drive/MyDrive/Colab Notebooks/통기계플젝/model/efficient{i}-fold_best-model.pth')  # 불러올 모델의 이름에서 fold 번호를 바꿔가면서 각 fold의 best model load 
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  model.to(device)
  model.eval() # model load 후, inference를 위한 eval 모드로 다시 설정

  predictions = []
  files = []
  score_list = []
  test_dataset = TestDataset(submission['image'], transforms_val)
  test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

  with torch.no_grad():
    for img_names, images in test_loader:
      images = images.to(device=device, dtype=dtype)
      scores = model(images)
      _, preds = scores.max(dim=1)
      
      files.extend(img_names)
      predictions.extend(preds.squeeze(0).detach().cpu().numpy())
      score_list.extend(scores.squeeze(0).detach().cpu().numpy()) ## score 형태 그대로 리스트에 저장/ 형태 [[3.XXXXXX, -3.XXXXXX], [..], ...]
  
  # 확률 평균 구하기 위해 매 fold 별로 sum 저장
  for i, score in enumerate(score_list):
    prob_zero[i] += softmax_0(score[0], score[1])



# Xception Net train

In [ ]:
num_epochs = 30
EARLY_STOPPING_EPOCH = 6
SEED = 42

# cross validation을 적용하기 위해 KFold 생성
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=SEED)

# train_df에서 train_idx와 val_idx를 생성
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(train_df),1): # enumerate( , 1) 1부터 index 시작
  print('[fold: %d]' % fold_index)

  # cuda cache 초기화
  torch.cuda.empty_cache()

  #train fold, validation fold 분할
  train = train_df.iloc[trn_idx]
  valid  = train_df.iloc[val_idx]

  tr_dataset = FaceDataset(image_label=train, transforms=transforms_tr)
  val_dataset = FaceDataset(image_label=valid, transforms=transforms_val)

  train_loader = DataLoader(tr_dataset, batch_size=64, shuffle=True, num_workers=8)
  valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=True, num_workers=8)

  # 모델 선언
  model = xceptionnet()
  model.to(device)

  # optimizer : RAdam / lr_scheduler : CosineAnnealing
  optimizer = optim.RAdam(model.parameters(), lr=0.0015, betas=(0.9, 0.999), weight_decay=1e-4)
  lr_sched = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


  # train 시작
  valid_early_stop = 0

  valid_best_loss = float('inf')
  since = time.time()

  final_train_loss = []
  final_train_acc = []
  final_valid_loss = []
  final_valid_acc = []

  for e in range(num_epochs) :
    print(f' ====================== epoch %d ======================' % (e+1) )
    train_loss_list = []
    train_acc_list = []

    # train
    for i, (images, targets) in enumerate(train_loader) : 
      optimizer.zero_grad()
      
      images = images.to(device, dtype)
      targets = targets.to(device, ltype)
      model.train()
    
      scores = model(images)
      _, preds = scores.max(dim=1)

      loss = F.cross_entropy(scores, targets)
      loss.backward()
      optimizer.step()
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      train_loss_list.append(loss)
      train_acc_list.append(acc)

      if i % 20 == 0 :
        print(f'Iteration %3.d | Train Loss  %.4f | Classifier Accuracy %2.2f' % (i, loss, acc))

    train_mean_loss = np.mean(train_loss_list, dtype="float64")
    train_mean_acc = np.mean(train_acc_list, dtype="float64")

    final_train_loss.append(train_mean_loss)
    final_train_acc.append(train_mean_acc)
    
    epoch_time = time.time() - since
    since = time.time()

    print('')
    print(f'[Summary] Elapsed time : %.0f m %.0f s' % (epoch_time // 60, epoch_time % 60))
    print(f'Train Loss Mean %.4f | Accuracy %2.2f ' % (train_mean_loss, train_mean_acc) )

    # validation 
    valid_loss_list = []
    valid_acc_list = []
   
    for i, (images, targets) in enumerate(valid_loader) : 
      optimizer.zero_grad()
      images = images.to(device=device, dtype=dtype)
      targets = targets.to(device=device, dtype=ltype)
      model.eval()

      with torch.no_grad():
        scores = model(images)
        loss = F.cross_entropy(scores, targets)
        _, preds = scores.max(dim=1)
      
      correct = sum(targets == preds).cpu()
      acc=(correct/64 * 100)

      valid_loss_list.append(loss)
      valid_acc_list.append(acc)
  
    val_mean_loss = np.mean(valid_loss_list, dtype="float64")
    val_mean_acc = np.mean(valid_acc_list, dtype="float64")

    final_valid_loss.append(val_mean_loss)
    final_valid_acc.append(val_mean_acc)

    print(f'Valid Loss Mean %.4f | Accuracy %2.2f ' % (val_mean_loss, val_mean_acc) )
    print('')

    if val_mean_loss < valid_best_loss:
      valid_best_loss = val_mean_loss
      valid_early_stop = 0
      # new best model save (valid 기준)
      best_model = model
      path = '/content/drive/MyDrive/Colab Notebooks/통기계플젝/model/xception/'
      torch.save({'model_state_dict' : best_model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict()}, f'{path}{fold_index}-fold_best-model.pth')

    else:
      # early stopping    
      valid_early_stop += 1
      if valid_early_stop >= EARLY_STOPPING_EPOCH:
        print("EARLY STOPPING!!")
        break

    lr_sched.step()
  best_models.append(best_model)
  if fold_index == 3:
    break

# Xception Net inference

In [ ]:
# inference에 사용할 model과 optimizer
#model = xceptionnet()
model = timm.create_model('xception', pretrained=False)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 2)
)
optimizer = optim.RAdam(model.parameters(), lr=0.0015, betas=(0.9, 0.999), weight_decay=1e-4)

for i in range(1, 4):
  checkpoint = torch.load(f'/content/drive/MyDrive/Colab Notebooks/통기계플젝/model/xception/{i}-fold_best-model.pth')  # 불러올 모델의 이름에서 fold 번호를 바꿔가면서 각 fold의 best model load 
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  model.to(device)
  model.eval() # model load 후, inference를 위한 eval 모드로 다시 설정

  predictions = []
  files = []
  score_list = []
  test_dataset = TestDataset(submission['image'], transforms_val)
  test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)


  with torch.no_grad():
    for img_names, images in test_loader:
      images = images.to(device=device, dtype=dtype)
      scores = model(images)
      _, preds = scores.max(dim=1)
      
      files.extend(img_names)
      predictions.extend(preds.squeeze(0).detach().cpu().numpy())
      score_list.extend(scores.squeeze(0).detach().cpu().numpy()) ## score 형태 그대로 리스트에 저장/ 형태 [[3.XXXXXX, -3.XXXXXX], [..], ...]
  
  # 확률 평균 구하기 위해 매 fold 별로 sum 저장
  for i, score in enumerate(score_list):
    prob_zero[i] += softmax_0(score[0], score[1])



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Soft voting

In [ ]:
# 평균과 예측값 저장
for i in range(5000):
  prob_zero[i] /= 6 # fold 개수로 나누어 확률 평균 계산
  prob_zero[i] = (prob_zero[i]<0.5)*1 # 0.5를 기준으로 예측값을 0과 1로 저장

In [ ]:
ensemble_sub = pd.DataFrame(columns=submission.columns)
ensemble_sub['image'] = files
ensemble_sub['label'] = prob_zero

In [ ]:
ensemble_sub.head()

,image,label
0,test14200.jpg,0
1,test12178.jpg,0
2,test12713.jpg,0
3,test13712.jpg,0
4,test11739.jpg,1


In [ ]:
ensemble_sub.to_csv("/content/drive/MyDrive/Colab Notebooks/통기계플젝/0601_EffNet4+XcpNet_soft.csv", index=False)